In [ ]:
pip install tensorflow


     -------------------------------------- 276.5/276.5 MB 2.4 MB/s eta 0:00:00
     ---------------------------------------- 5.6/5.6 MB 6.8 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 7.2 MB/s eta 0:00:00
     ---------------------------------------- 4.3/4.3 MB 8.1 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 kB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 7.9 MB/s eta 0:00:00
     -------------------------------------- 430.4/430.4 kB 8.9 MB/s eta 0:00:00
     -------------------------------------- 440.8/440.8 kB 6.8 MB/s eta 0:00:00
     -------------------------------------- 126.5/126.5 kB 7.8 MB/s eta 0:00:00
     ---------------------------------------- 65.5/65.5 kB 3.7 MB/s eta 0:00:00
     ---------------------------------------- 24.4/24.4 MB 9.6 MB/s eta 0:00:00
     ------------------------------------- 181.8/181.8 kB 11.4 MB/s eta 0:00:00
     -------------------------------------- 151.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout

In [ ]:
true_data = pd.read_csv('True.csv')
true_data['label'] = 1  # Assign label 1 for true news



In [ ]:
# Load the fake news dataset
fake_data = pd.read_csv('Fake.csv')
fake_data['label'] = 0  # Assign label 0 for fake news

In [ ]:
# Combine the datasets
data = pd.concat([true_data, fake_data], ignore_index=True)

In [ ]:
data.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [ ]:
X = data['text']
y = data['label']

In [ ]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [ ]:
def preprocess_text(text):
    text = text.lower()
    text = ' '.join(word for word in text.split() if word.isalnum())
    text = ' '.join(word for word in text.split() if word not in ENGLISH_STOP_WORDS)
    return text

In [ ]:
X = X.apply(preprocess_text)


In [ ]:
for idx, text in enumerate(X):
    preprocessed_text = preprocess_text(text)
    print(f"Original text {idx + 1}:\n{text}\nPreprocessed text {idx + 1}:\n{preprocessed_text}\n")
    X[idx] = preprocessed_text

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Original text 44896:
patrick henningsen 21st century wireremember obama administration told world hoped identify reliable moderate rebels hanging turkey want fight washington drama infamous train equip program create arab army want pentagon officials announced new plan train moderate time project place inside syria hell international told requested nato allies singing hymn claiming new effort help securing turkey s porous border story washington s political cover fashioned popular protect civilized europe invading hordes terrorists hide stated wall street program address tense dialogue washington ankara sealing western governments long complained avenue extremists leave syria travel timing announcement uncanny backdrop unlikely isis suicide bomb terrorist victims just happened 14 german feeding perfectly europe s new binary fear set terrorism shades undoubtedly won t western journalists entertaining conspiracy friends nation builders banking global public suffering syria time really pa

In [ ]:
max_words = 10000
max_len = 200
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)


In [ ]:
model = Sequential()
model.add(Embedding(max_words, 128, input_length=max_len))
model.add(SimpleRNN(64, return_sequences=True))
model.add(SimpleRNN(32))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
449/449 [==============================] - 31s 67ms/step - loss: 0.6949 - accuracy: 0.5201 - val_loss: 0.6923 - val_accuracy: 0.5193
Epoch 2/5
449/449 [==============================] - 30s 66ms/step - loss: 0.6750 - accuracy: 0.5726 - val_loss: 0.6510 - val_accuracy: 0.6041
Epoch 3/5
449/449 [==============================] - 30s 66ms/step - loss: 0.6297 - accuracy: 0.6310 - val_loss: 0.5258 - val_accuracy: 0.7149
Epoch 4/5
449/449 [==============================] - 30s 67ms/step - loss: 0.5757 - accuracy: 0.6640 - val_loss: 0.5107 - val_accuracy: 0.7468
Epoch 5/5
449/449 [==============================] - 30s 66ms/step - loss: 0.5563 - accuracy: 0.6967 - val_loss: 0.5798 - val_accuracy: 0.7311


In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")


281/281 [==============================] - 3s 12ms/step - loss: 0.5844 - accuracy: 0.7262
Test Loss: 0.5844, Test Accuracy: 0.7262
